# Submitted By:
Gal Ben Lulu - 314668393
  |  Rotem Barel - 318223872

## Import libraries

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd
import smtplib
import csv
import os
import re
from IPython.display import display
from datetime import datetime, timedelta
from collections import defaultdict

# Functions

### Process_url_request Function

The `process_url_request` function fetches the HTML content from a given URL using the `requests` module and creates a BeautifulSoup object (`soup`) for parsing and extracting data from the webpage.


In [257]:
def process_url_request(website_url):
    """
    This function process provided URL get its data using requets module
    and contrunct soup data using BeautifulSoup for scarping
    """
    requets_data = requests.get(website_url)
    if requets_data.status_code == 200:
        soup = BeautifulSoup(requets_data.text,'html')
        return soup
    return None

In [258]:
def convert_to_datetime(date_string):
    date_format = "%d/%m/%Y"
    return datetime.strptime(date_string, date_format)

### Last_day_of_month
The function `last_day_of_month` calculates the last day of a given month and year by determining the first day of the next month and subtracting one day from it using Python's `datetime` and `timedelta` modules.

In [259]:
def last_day_of_month(year, month):
    if month == 12:  
        next_month = datetime(year + 1, 1, 1)
    else:
        next_month = datetime(year, month + 1, 1)
    return next_month - timedelta(days=1)

### Days_until_end_of_month
The function calculates the number of days remaining until the end of a specified month and year based on the provided date string in "month/year" format. It extracts the month and year from the string, computes the last day of that month using `last_day_of_month`, and then returns the difference in days between today's date and the calculated last day of the month.


In [260]:
def days_until_end_of_month(test_date_str):
    if pd.isna(test_date_str) or not isinstance(test_date_str, str):
        return None
    
    month, year = map(int, test_date_str.split('/'))
    last_day = last_day_of_month(year, month)
    today = datetime.today()
    return (last_day - today).days

### Get_table_data
extracts data from an HTML table by iterating through each row (`<tr>`) and extracting the text content of each cell (`<td>`). It constructs a dictionary where each key-value pair corresponds to the stripped text content of the first and second cells in each row respectively, but only if there are exactly two cells in the row.


In [261]:
def get_table_data(table):
    data = {}
    for row in table.find_all('tr'):
        cols = row.find_all('td')
        if len(cols) == 2:
            key = cols[0].text.strip()
            value = cols[1].text.strip()
            data[key] = value
    return data

### Fetch_citroen_data
retrieves data from an API endpoint specified by `url` using parameters such as `resource_id`, `limit`, `offset`, and `q`. It repeatedly sends GET requests with increasing offsets until all records containing the keyword "סיטרואן" (Citroen in Hebrew) are fetched. The function accumulates all retrieved records into a list `all_records` and returns it once all pages of data have been fetched.


In [262]:
def fetch_citroen_data(url, resource_id):
    limit = 1000  # מספר הרשומות בכל קריאה
    offset = 0
    all_records = []

    while True:
        response = requests.get(url, params={
            'resource_id': resource_id, 
            'limit': limit, 
            'offset': offset, 
            'q': 'סיטרואן'
        })
        data = response.json()

        if 'result' in data and 'records' in data['result']:
            records = data['result']['records']
            all_records.extend(records)

            if len(records) < limit:
                break  # סיום הפאגינציה אם יש פחות רשומות ממספר הרשומות המבוקש
            else:
                offset += limit
        else:
            break  # סיום אם אין נתונים נוספים

    return all_records


### Clean_model_name
removes specific patterns and characters from the input `model_name` string used in dataset `data2`. It removes variations of "דל5" or "דלת5" in Hebrew, removes trailing single quotes (`'`), strips Hebrew characters, eliminates numbers with decimal points, and removes instances of "5 '" and "' 5".


In [263]:
# פונקציה למיפוי שמות הדגמים בדיוק מלא לפי רשימת data1
def map_exact_model_name(model_name):
    for name in data1:
        if re.search(r'\b' + re.escape(name) + r'\b', model_name):
            return name
    return model_name


In [264]:
# פונקציה לניקוי שמות הדגמים ב-data 2 מהחלקים "דל5" או "דלת5" בעברית והתו "'" בסוף ולהסרת מספרים עם נקודה
def clean_model_name(model_name):
    if isinstance(model_name, str):
        model_name = re.sub(r'(דלת?5|5דלת|\'דלת5)\b', '', model_name).strip()
        model_name = re.sub(r'\s+', ' ', model_name)
        model_name = re.sub(r'[\u0590-\u05FF]+', '', model_name)
        model_name = re.sub(r'\b\d+\.\d+\b', '', model_name)  # הסרת מספרים עם נקודה
        model_name = re.sub(r"5\s*'", '', model_name).strip()  # הסרת "5 '"
        model_name = re.sub(r"'\s*5", '', model_name).strip()  # הסרת "' 5"
        model_name = model_name.rstrip("'")  # הסרת "'" מהסוף של המחרוזת
    return model_name

In [265]:
url = "https://data.gov.il/api/3/action/datastore_search"
resource_id = "5e87a7a1-2f6f-41c1-8aec-7216d52a6cf6"

citroen_records = fetch_citroen_data(url, resource_id)
df_citroen = pd.DataFrame.from_records(citroen_records)

display(df_citroen)

,_id,sug_degem,tozeret_cd,tozeret_nm,tozeret_eretz_nm,tozar,degem_cd,degem_nm,shnat_yitzur,mispar_rechavim_pailim,mispar_rechavim_le_pailim,kinuy_mishari,rank
0,20907,P,817,סיטרואן צרפת,צרפת,סיטרואן,2050,X16C00,1997,0,4,סיטרואן 8.1 XS AITNA,0.076422
1,8483,P,817,סיטרואן צרפת,צרפת,סיטרואן,1101,N2LFZM,1999,0,33,סיטרואן 8.1 XS ARASX,0.076422
2,18211,P,817,סיטרואן צרפת,צרפת,סיטרואן,1300,S1KFXF,1999,0,5,סיטרואן 4.1 XS OXAS,0.076422
3,78808,P,817,סיטרואן צרפת,צרפת,סיטרואן,1211,X1RFVM,1999,0,0,סיטרואן 0.2 CXE AITN,0.076422
4,10029,P,817,סיטרואן צרפת,צרפת,סיטרואן,1190,N2DJYF,1999,0,36,סיטרואן 9.1 DX ARASX,0.076422
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2469,2183,P,817,סיטרואן צרפת,צרפת,סיטרואן,60,LARFKC,2006,6,12,C4,0.070624
2470,1724,P,496,סיטרואן ספרד,ספרד,סיטרואן,83,3DBHZT/2SM,2018,4,1,C4 PICASSO,0.070624
2471,2474,P,496,סיטרואן ספרד,ספרד,סיטרואן,91,UARFJF,2008,0,1,C4 PICASSO,0.070624
2472,428,P,817,סיטרואן צרפת,צרפת,סיטרואן,155,ACYHZR,2019,268,9,C5 AIRCROSS,0.070624


In [266]:
df_citroen["kinuy_mishari"].unique()

array(['סיטרואן 8.1 XS AITNA', 'סיטרואן 8.1 XS ARASX',
       'סיטרואן 4.1 XS OXAS', 'סיטרואן 0.2 CXE AITN',
       'סיטרואן 9.1 DX ARASX', 'סיטרואן 9.1 DTXS AIT', 'C4 PICASSO',
       'XSARA PICASSO', 'BERLINGO', 'C-ELYSEE', 'XSARA 2', 'SAXOSX1.4',
       'C5 AIRCROSS', 'C4', 'SAXO', 'XSARA', "XE 6V MX 5 דל' אוט'",
       'JUMPY', 'C3 PICASSO', 'C3', 'C1', 'C5', 'SAXOX 1.1',
       'C4 SPACETOURER', 'C6', '9.1 PUS OGNILREB דלת', 'DS3',
       'GRAND BERLING0', 'DS4', "0.2 ENACLOV XZ 5 דל'",
       "0.2 XSV AITNAX 5 דל'", 'JUMPY HDI', 'XANTIA', 'BERLINGO_SX 1.9',
       "0.2 ENACLOV XZ 3 דל'", '0.3 CXE MX 5דלת אוטו',
       "TC 0.2 XSV MX 5דל'", '5 דל MX TCT 0.2 CXE', 'C4  PICASSO',
       'C4 CACTUS', 'JUMPY  HDI', 'C8', "MX פרטי 5דלת'",
       "D9.1XSXZ 5דלתו'", 'C3 PICASS0', 'SAXOSX 1.4', 'ZXSX1.9D', 'DS5',
       'פרטי דו שימושי 5 דז', 'NEMO', 'BERLINGO X 1.9', 'C4 GD PICASSO',
       '4.1 TNET XZ 5דלת 5 ה', 'D9.1 EGATNAVA XZ דיז',
       'D9.1 EUGUF XZ דיזל ס', 'C2', 'C3 AIRCROSS'

### Calculate_supply_index
calculates a supply score for each row in the DataFrame `df` based on matching criteria with data from `df_citroen`. It initializes a new column 'Supply_score' with an initial value of 0. For each row in `df`, it retrieves the manufacturer, model, and year of the car. It then filters `df_citroen` to find rows that match these criteria (manufacturer, exact model cleaned, and year). It calculates the sum of 'mispar_rechavim_pailim' (active vehicles count) for these matching rows and assigns this sum as the 'Supply_score' for the corresponding row in `df`. Finally, it returns the updated `df` with the supply scores.


In [290]:
def calculate_supply_index(df, df_citroen):
    # יצירת עמודה חדשה במדדים בשם 'supply_index' עם ערך התחלתי 0
    df['Supply_score'] = 0
    
    for index, row in df.iterrows():
        manufacturer = row['Manufactor']
        model = row['Model']
        year = row['Year']
        
        # חישוב סך כל הרכבים הפעילים בטבלת משרד התחבורה שמקיימים את התנאים
        matching_rows = df_citroen[
            (df_citroen['tozar'] == manufacturer) &
            (df_citroen['exact_model_mapped_cleaned'] == model) &
            (df_citroen['shnat_yitzur'] == year)
        ]
        
        # חישוב סך כל הרכבים הפעילים
        total_active_vehicles = matching_rows['mispar_rechavim_pailim'].sum()

        
        # הוספת ערך מספר הרכבים הפעילים לשורה בטבלת הנתונים
        df.at[index, 'Supply_score'] = total_active_vehicles
        
    return df 

### Parse_ad_data
function extracts and structures data from an advertisement (`ad`) on a website. It retrieves the ad's title, separates it into manufacturer and model components, and constructs a URL from the ad's link. It then fetches and parses the webpage content using `process_url_request`. If successful, it gathers information such as price, additional details from a table if present, and metadata like the number of pictures, creation date, last bump date, and description from the ad's body. This information is compiled into a dictionary (`data`) and returned.


In [291]:
def parse_ad_data(ad):
    title = ad.find('h2', class_='card-title').get_text()
    Manufacturer = title.split()
    link = ad.find('a')['href']
    url = f'https://www.ad.co.il{link}'
    
    ad_soup = process_url_request(url)
    if not ad_soup:
        return None

    adBody = ad_soup.find('div', class_="col-xxl-4 col-lg-4 col-md-5")
    
    price_elements = adBody.find_all('h2', class_='card-title')
    price = price_elements[1].text.strip() if len(price_elements) > 1 else 'N/A'
    
    data = {
        'Manufactor': Manufacturer[0],
        'Model': Manufacturer[1],
        'Price': price
    }

    table = adBody.find('table', class_='table table-sm mb-4') if adBody else None
    if table:
        table_data = get_table_data(table)
        data.update(table_data)
    
    adLeft = ad_soup.find('div', class_="col-xxl-8 col-lg-8 col-md-7")
    if adLeft:
        NumOfPictures = adLeft.find_all('div', class_="justify-content-center px-1")
        data['Num_of_pictures'] = len(NumOfPictures)

        Dates = adLeft.find_all('div', class_='px-3')
        if len(Dates) > 0:
            data['Create_date'] = Dates[0].text.split()[2]
        if len(Dates) > 1:
            data['Last_bump_date'] = Dates[1].text.split()[3]
        
        description = adLeft.find('p', class_='text-word-break')
        data['Description'] = description.text.strip() if description else 'N/A'

    return data


In [292]:
base_url = 'https://www.ad.co.il/car?sp261=13899&pageindex='
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}


all_ads_data = []
page = 1
while True:
    soup = process_url_request(base_url + str(page))
    if not soup:
        break

    cars = soup.find('div', class_='cards-wrap s m l')
    carAd = cars.find_all('div', class_='card-body p-md-3') if cars else []

    if not carAd:
        break

    for ad in carAd:
        
        ad_data = parse_ad_data(ad)
        
        if ad_data:
            all_ads_data.append(ad_data)

    page += 1

# יצירת DataFrame 
df = pd.DataFrame(all_ads_data)
column_mapping_english = {
    'שנה': 'Year',
    'יד': 'Hand',
    'ת. הילוכים': 'Gear',
    'נפח': 'Engine_capacity',
    'סוג מנוע': 'Engine_type',
    'בעלות קודמת': 'Prev_ownership',
    'בעלות נוכחית': 'Curr_ownership',
    'אזור': 'Area',
    'עיר': 'City',
    'Price': 'Price',
    'Num_of_pictures': 'Pic_num',
    'Create_date': 'Cre_date',
    'Last_bump_date': 'Repub_date',
    'Description': 'Description',
    'צבע': 'Color',
    'ק"מ': 'Km',
    'טסט עד': 'Test'
}

df.rename(columns=column_mapping_english, inplace=True)
df['Test'] = df['Test'].apply(days_until_end_of_month)
                                                                                               
new_order =['Manufactor', 'Year','Model', 'Hand', 'Gear', 'Engine_capacity', 'Engine_type', 'Prev_ownership', 'Curr_ownership', 'Area', 'City', 'Price', 'Pic_num', 'Cre_date', 'Repub_date', 'Description', 'Color', 'Km', 'Test']
df = df[new_order]
# הצגת הנתונים
display(df)


,Manufactor,Year,Model,Hand,Gear,Engine_capacity,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test
0,סיטרואן,2023,C5,1,אוטומטית,"1,600",בנזין,פרטית,פרטית,פתח תקוה והסביבה,פתח תקווה,"165,000 ₪",8,19/05/2024,21/05/2024,סיטרואן סי5 איירקרוס שנת 2023 עם 8600 ק״מ. הדג...,כחול כהה מטאלי,"8,600",291.0
1,סיטרואן,2019,DS3,1,אוטומטית,"1,200",היבריד,פרטית,פרטית,מושבים בשרון,שפיים,"95,000 ₪",4,12/05/2024,12/05/2024,רכב שמור מאוד,סגול,"70,000",NaN
2,סיטרואן,2022,DS3,1,אוטומטית,"1,100",חשמלי,אחר,פרטית,גדרה יבנה והסביבה,גדרה,"139,000 ₪",5,12/05/2024,12/05/2024,רכב חשמלי 330 קמ. שמור. שימש כרכב משני. נהיגה ...,לבן מטאלי,"27,400",NaN
3,סיטרואן,2012,C4,3,אוטומטית,"1,600",בנזין,פרטית,פרטית,ראש העין והסביבה,ראש העין,"3,500 ₪",4,09/04/2024,28/04/2024,רכב מושלם לחלקים רכב שלם יפה במצב נסיעה בצבע ש...,שמפניה,"160,000",NaN
4,סיטרואן,2017,C4,2,אוטומטית,"1,600",בנזין,פרטית,פרטית,באר שבע והסביבה,באר שבע,"45,000 ₪",4,04/10/2023,25/04/2024,סיטרואן C4 פיקאסו 2017\nאוטומט-בנזין-5 דלתות-7...,סגול,"180,000",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,סיטרואן,2019,C4,2,אוטומטית,"1,199",בנזין,NaN,NaN,נס ציונה - רחובות,רחובות,"88,000 ₪",0,22/05/2022,22/05/2022,כמו חדש מוחלף בגלל קשיי נגישות לאדם מבוגר\r\nמ...,NaN,"32,000",NaN
86,סיטרואן,2018,C3,1,אוטומטית,"1,200",בנזין,NaN,NaN,"ראשל""צ והסביבה",ראשון לציון,"55,900 ₪",0,29/04/2022,29/04/2022,לבוא לקחת. דו צבעי- כסוף עם גג אדום. התחייבות ...,NaN,"104,300",NaN
87,סיטרואן,2018,C3,1,אוטומטית,"1,200",בנזין,NaN,NaN,חיפה וחוף הכרמל,חיפה,"61,000 ₪",0,28/04/2022,28/04/2022,N/A,NaN,"41,000",NaN
88,סיטרואן,2018,C1,1,רובוטית,800,בנזין,פרטית,פרטית,NaN,דבוריה,"41,000 ₪",0,12/01/2022,12/01/2022,תוספות\r\n\t\t\t\tהכל תוספות,לבן,"52,800",NaN


In [293]:
# יצירת DataFrame עבור הדגמים
#data1 = df['Model'].unique()
#data2 = df_citroen["kinuy_mishari"].unique()
#df_citroen = pd.DataFrame({'kinuy_mishari': data2})


# יצירת עמודה חדשה ב DataFrame עבור הדגמים הממופים לפי המיפוי המדויק
df_citroen['exact_model_mapped'] = df_citroen['kinuy_mishari'].apply(map_exact_model_name)

# ניקוי שמות הדגמים ב-data 2 מהחלקים "דל5" או "דלת5", המספרים עם נקודה והתו "'" בסוף
df_citroen['exact_model_mapped_cleaned'] = df_citroen['exact_model_mapped'].apply(clean_model_name)

# הדפסת כל הערכים הייחודיים עם הערך המקורי והניקוי
for idx, row in df_citroen.iterrows():
    original = row['exact_model_mapped']
    cleaned = row['exact_model_mapped_cleaned']
    print(f"Original: {original} | Cleaned: {cleaned}")


Original: סיטרואן 8.1 XS AITNA | Cleaned: XS AITNA
Original: סיטרואן 8.1 XS ARASX | Cleaned: XS ARASX
Original: סיטרואן 4.1 XS OXAS | Cleaned: XS OXAS
Original: סיטרואן 0.2 CXE AITN | Cleaned: CXE AITN
Original: סיטרואן 9.1 DX ARASX | Cleaned: DX ARASX
Original: סיטרואן 0.2 CXE AITN | Cleaned: CXE AITN
Original: סיטרואן 9.1 DTXS AIT | Cleaned: DTXS AIT
Original: סיטרואן 0.2 CXE AITN | Cleaned: CXE AITN
Original: סיטרואן 8.1 XS AITNA | Cleaned: XS AITNA
Original: C4 | Cleaned: C4
Original: XSARA PICASSO | Cleaned: XSARA PICASSO
Original: BERLINGO | Cleaned: BERLINGO
Original: C | Cleaned: C
Original: XSARA 2 | Cleaned: XSARA 2
Original: C4 | Cleaned: C4
Original: BERLINGO | Cleaned: BERLINGO
Original: SAXOSX1.4 | Cleaned: SAXOSX1.4
Original: C5 | Cleaned: C5
Original: C4 | Cleaned: C4
Original: SAXO | Cleaned: SAXO
Original: XSARA | Cleaned: XSARA
Original: SAXOSX1.4 | Cleaned: SAXOSX1.4
Original: XE 6V MX 5 דל' אוט' | Cleaned: XE 6V MX  
Original: JUMPY | Cleaned: JUMPY
Original: C3 | 

In [294]:
df['Model'].unique()

array(['C5', 'DS3', 'C4', 'DS4', 'C3', 'C', 'C1', 'קסרה', 'AX', 'CX',
       'C8', 'C6'], dtype=object)

In [285]:
df_citroen["exact_model_mapped_cleaned"].unique()

array(['XS AITNA', 'XS ARASX', 'XS OXAS', 'CXE AITN', 'DX ARASX',
       'DTXS AIT', 'C4', 'XSARA PICASSO', 'BERLINGO', 'C', 'XSARA 2',
       'SAXOSX1.4', 'C5', 'SAXO', 'XSARA', 'XE 6V MX  ', 'JUMPY', 'C3',
       'C1', 'SAXOX', 'C6', 'PUS OGNILREB', 'DS3', 'GRAND BERLING0',
       'DS4', 'ENACLOV XZ', 'XSV AITNAX', 'JUMPY HDI', 'XANTIA',
       'BERLINGO_SX', 'ENACLOV XZ 3 ', 'CXE MX', 'TC  XSV MX',
       '5  MX TCT  CXE', 'C8', 'MX  ', 'D9.1XSXZ', 'SAXOSX', 'ZXSX1.9D',
       'DS5', '5', 'NEMO', 'BERLINGO X', 'TNET XZ 5', 'D9.1 EGATNAVA XZ',
       'D9.1 EUGUF XZ', 'C2', 'ARUA XZ  5', '4.1XS OXAS', 'ENACIOV XZ',
       'JUMP', 'XELFER XZ', 'ZXSX 1.9D', 'C15', 'SAX0 VTR1.6',
       'EGAMI XZ  5', '', 'XELFER XZ 3 ', 'TNAX', 'OIRUF XZ 3', 'XANIA',
       'BERLINGO SUPER', 'SAXOVTS1.6-16V', 'XS TNAX', 'BERLINGO SX',
       'C4X', 'JUMPY VP', 'C4PICASSO', '9.1_DT YPM', 'JUMPY TD',
       'XEIFER XZ', 'SAXO X1.1', 'XS AINAX 5', 'XS AITNAX', 'XMEXCLUSTCT',
       'BERLING0', 'JUMP HDI', 

In [286]:
print(df.dtypes)

Manufactor          object
Year                object
Model               object
Hand                object
Gear                object
Engine_capacity     object
Engine_type         object
Prev_ownership      object
Curr_ownership      object
Area                object
City                object
Price               object
Pic_num              int64
Cre_date            object
Repub_date          object
Description         object
Color               object
Km                  object
Test               float64
dtype: object


In [287]:
df['Manufactor'] = df['Manufactor'].astype(str)
df['Year'] = pd.to_numeric(df['Year'], errors='coerce').astype('Int64')
df['Model'] = df['Model'].astype(str)
df['Hand'] = pd.to_numeric(df['Hand'], errors='coerce').astype('Int64')
df['Gear'] = pd.Categorical(df['Gear'])
df['Engine_capacity'] = pd.to_numeric(df['Engine_capacity'].str.replace(',', ''), errors='coerce').astype('Int64')
df['Engine_type'] = pd.Categorical(df['Engine_type'])
df['Prev_ownership'] = pd.Categorical(df['Prev_ownership'])
df['Curr_ownership'] = pd.Categorical(df['Curr_ownership'])
df['Area'] = df['Area'].astype(str, errors='ignore')
df['City'] = df['City'].astype(str)
df['Price'] = pd.to_numeric(df['Price'].str.replace('[₪,]', '', regex=True), errors='coerce').astype(float)
df['Pic_num'] = df['Pic_num'].astype(int)
df['Cre_date'] = df['Cre_date'].apply(convert_to_datetime)
df['Repub_date'] = df['Repub_date'].apply(convert_to_datetime)
df['Description'] = df['Description'].astype(str)
df['Color'] = df['Color'].astype(str)
df['Km'] = pd.to_numeric(df['Km'].str.replace(',', ''), errors='coerce').astype('Int64')
df['Test'] = pd.to_numeric(df['Test'], errors='coerce').astype('Int64')

print(df.dtypes)

Manufactor                 object
Year                        Int64
Model                      object
Hand                        Int64
Gear                     category
Engine_capacity             Int64
Engine_type              category
Prev_ownership           category
Curr_ownership           category
Area                       object
City                       object
Price                     float64
Pic_num                     int32
Cre_date           datetime64[ns]
Repub_date         datetime64[ns]
Description                object
Color                      object
Km                          Int64
Test                        Int64
dtype: object


In [289]:
#df_citroen['tozar'] = df_citroen['tozar'].str.strip()
#df_citroen['kinuy_mishari'] = df_citroen['kinuy_mishari'].astype(str)
#df_citroen['shnat_yitzur'] = pd.to_numeric(df_citroen['shnat_yitzur'], errors='coerce').astype('Int64')
df = calculate_supply_index(df, df_citroen)
df['Supply_score'] = df['Supply_score'].astype('Int64')
df

,Manufactor,Year,Model,Hand,Gear,Engine_capacity,Engine_type,Prev_ownership,Curr_ownership,Area,...,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test,Supply_score,supply_score
0,סיטרואן,2023,C5,1,אוטומטית,1600,בנזין,פרטית,פרטית,פתח תקוה והסביבה,...,165000.0,8,2024-05-19,2024-05-21,סיטרואן סי5 איירקרוס שנת 2023 עם 8600 ק״מ. הדג...,כחול כהה מטאלי,8600,291,0,856.0
1,סיטרואן,2019,DS3,1,אוטומטית,1200,היבריד,פרטית,פרטית,מושבים בשרון,...,95000.0,4,2024-05-12,2024-05-12,רכב שמור מאוד,סגול,70000,<NA>,0,0.0
2,סיטרואן,2022,DS3,1,אוטומטית,1100,חשמלי,אחר,פרטית,גדרה יבנה והסביבה,...,139000.0,5,2024-05-12,2024-05-12,רכב חשמלי 330 קמ. שמור. שימש כרכב משני. נהיגה ...,לבן מטאלי,27400,<NA>,0,0.0
3,סיטרואן,2012,C4,3,אוטומטית,1600,בנזין,פרטית,פרטית,ראש העין והסביבה,...,3500.0,4,2024-04-09,2024-04-28,רכב מושלם לחלקים רכב שלם יפה במצב נסיעה בצבע ש...,שמפניה,160000,<NA>,0,888.0
4,סיטרואן,2017,C4,2,אוטומטית,1600,בנזין,פרטית,פרטית,באר שבע והסביבה,...,45000.0,4,2023-10-04,2024-04-25,סיטרואן C4 פיקאסו 2017\nאוטומט-בנזין-5 דלתות-7...,סגול,180000,<NA>,0,1619.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,סיטרואן,2019,C4,2,אוטומטית,1199,בנזין,NaN,NaN,נס ציונה - רחובות,...,88000.0,0,2022-05-22,2022-05-22,כמו חדש מוחלף בגלל קשיי נגישות לאדם מבוגר\r\nמ...,nan,32000,<NA>,0,801.0
86,סיטרואן,2018,C3,1,אוטומטית,1200,בנזין,NaN,NaN,"ראשל""צ והסביבה",...,55900.0,0,2022-04-29,2022-04-29,לבוא לקחת. דו צבעי- כסוף עם גג אדום. התחייבות ...,nan,104300,<NA>,0,1254.0
87,סיטרואן,2018,C3,1,אוטומטית,1200,בנזין,NaN,NaN,חיפה וחוף הכרמל,...,61000.0,0,2022-04-28,2022-04-28,N/A,nan,41000,<NA>,0,1254.0
88,סיטרואן,2018,C1,1,רובוטית,800,בנזין,פרטית,פרטית,nan,...,41000.0,0,2022-01-12,2022-01-12,תוספות\r\n\t\t\t\tהכל תוספות,לבן,52800,<NA>,0,336.0


In [ ]:
df_citroen['exact_model_mapped_cleaned'].unique()

In [ ]:
folder_path = r"C:\Users\Rotem\Desktop\חומרי לימוד תואר תעשייה וניהול\שנה ג\סמסטר ב\כרייה וניתוח נתונים מתקדם פייתון"
csv_path = os.path.join(folder_path, "ad_data.csv")
df.to_csv(csv_path, index=False, encoding='utf-8-sig')